In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import os

In [ ]:
# Define the training and test data directories
train_dir = "data/train"
test_dir = "data/test"

# Define the classes
classes = ["A", "R"]

weights = [1, 10]  # Emphesize on R category

# Define the transform to be applied to the input images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
# Load the training data
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=2)

# Load the test data
test_dataset = torchvision.datasets.ImageFolder(test_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=2)

# Load the ResNet-18 model
model = torchvision.models.resnet18(weights='ResNet18_Weights.DEFAULT')

# Replace the last fully connected layer with a new layer with 1 output classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

# Define the loss function and optimizer
pos_weight = torch.tensor([10.0])
loss_fn =  nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = labels.unsqueeze(1)
        loss = loss_fn(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 9:    # Print every 10 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 200:.3f}")
            running_loss = 0.0

print("Training complete.")

In [ ]:
# Test the model
correct = 0
total = 0
test_labels = []
test_probabilities = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        logits = model(images)
        predicted = torch.sigmoid(logits)
        test_labels += list(labels.numpy())
        test_probabilities += list(predicted.numpy())

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Calculate fpr, tpr and thresholds
fpr, tpr, thresholds = roc_curve(test_labels, test_probabilities)

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # plot diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Probabilities of more than 0.5 will be classified as R
# The lower value will increase recall while higer vale will favor precision
threshold = 0.5

In [ ]:
test_predictions = []
for l, p in zip(test_labels, test_probabilities):
    test_predictions.append(int(p>threshold))

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score


# Compute precision and recall
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

# Print results
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')